In [1]:
using Graphs
using Printf
using CSV
using DataFrames

In [17]:
"""
    write_gph(dag::DiGraph, idx2names, filename)

Takes a DiGraph, a Dict of index to names and a output filename to write the graph in `gph` format.
"""
function write_gph(dag::DiGraph, idx2names, filename)
    open(filename, "w") do io
        for edge in edges(dag)
            @printf(io, "%s,%s\n", idx2names[src(edge)], idx2names[dst(edge)])
        end
    end
end


function compute(infile, outfile)
    df = CSV.read("example.csv", DataFrame)

    # WRITE YOUR CODE HERE
    # FEEL FREE TO CHANGE ANYTHING ANYWHERE IN THE CODE
    # THIS INCLUDES CHANGING THE FUNCTION NAMES, MAKING THE CODE MODULAR, BASICALLY ANYTHING

end

#################### parameter-learning 1
function sub2ind(siz, x)
    k = vcat(1, cumprod(siz[1:end-1]))
    return dot(k, x .- 1) + 1
end

function statistics(vars, G, D::Matrix{Int})
    n = size(D, 1)
    r = [vars[i].r for i in 1:n]
    q = [prod([r[j] for j in inneighbors(G,i)]) for i in 1:n]
    M = [zeros(q[i], r[i]) for i in 1:n]
    for o in eachcol(D)
        for i in 1:n
            k = o[i]
            parents = inneighbors(G,i)
            j = 1
            if !isempty(parents)
                 j = sub2ind(r[parents], o[parents])
            end
            M[i][j,k] += 1.0
        end
    end
    return M
end
####################

#################### parameter-learning 2
function prior(vars, G)
    n = length(vars)
    r = [vars[i].r for i in 1:n]
    q = [prod([r[j] for j in inneighbors(G,i)]) for i in 1:n]
    return [ones(q[i], r[i]) for i in 1:n]
end
####################

#################### structure-learning 1
function bayesian_score_component(M, α)
    p =  sum(loggamma.(α + M))
    p -= sum(loggamma.(α))
    p += sum(loggamma.(sum(α,dims=2)))
    p -= sum(loggamma.(sum(α,dims=2) + sum(M,dims=2)))
    return p
end

function bayesian_score(vars, G, D)
    n = length(vars)
    M = statistics(vars, G, D)
    α = prior(vars, G)
    return sum(bayesian_score_component(M[i], α[i]) for i in 1:n)
end
####################

#################### structure-learning 2
struct K2Search
    ordering::Vector{Int} # variable ordering
end

function fit(method::K2Search, vars, D)
    G = SimpleDiGraph(length(vars))
    for (k,i) in enumerate(method.ordering[2:end])
        y = bayesian_score(vars, G, D)
        while true
            y_best, j_best = -Inf, 0
            for j in method.ordering[1:k]
                if !has_edge(G, j, i)
                    add_edge!(G, j, i)
                    y′ = bayesian_score(vars, G, D)
                    if y′ > y_best
                        y_best, j_best = y′, j
                    end
                    rem_edge!(G, j, i)
                end
            end
            if y_best > y
                y = y_best
                add_edge!(G, j_best, i)
            else
                break
            end
        end
    end
    return G
end
####################

fit (generic function with 1 method)

In [10]:
df = CSV.read("example/example.csv", DataFrame)
data = transpose(Matrix(df))

6×20 transpose(::Matrix{Int64}) with eltype Int64:
 3  1  3  1  2  1  1  3  1  3  2  2  2  2  1  2  1  3  3  1
 3  3  1  3  3  3  3  3  3  1  3  3  2  3  1  2  2  1  1  3
 2  2  1  1  3  1  3  2  1  1  3  2  3  3  2  1  1  1  1  1
 3  3  2  2  2  1  1  3  3  3  3  3  1  3  3  3  3  3  3  2
 1  2  3  3  2  1  2  1  2  3  2  3  1  2  3  2  2  1  3  3
 3  3  1  2  1  3  3  3  1  3  3  3  3  3  2  3  3  2  3  1

In [14]:
names(df)

6-element Vector{String}:
 "parent1"
 "child1"
 "parent2"
 "child2"
 "parent3"
 "child3"

In [16]:
fit(K2Search(1:4), [Variable(2) for i in 1:4], data)

LoadError: UndefVarError: Variable not defined

In [23]:
graph = Graphs.loadgraph("example/example.gph")

LoadError: BoundsError: attempt to access 2-element Vector{SubString{String}} at index [3]

In [20]:

bayesian_score()

LoadError: syntax: character literal contains multiple characters